文章：[推荐系统遇上深度学习(四)--多值离散特征的embedding解决方案](https://blog.csdn.net/jiangjiang_jian/article/details/80631023)

在处理DeepFM数据时，由于每一个离散特征只有一个取值，因此我们在处理的过程中，将原始数据处理成了两个文件，一个记录特征的索引，一个记录了特征的值，而每一列，则代表一个离散特征。

但假如，我们某一个离散特征有多个取值呢？举个例子来说，每个人喜欢的NBA球队，有的人可能喜欢火箭和湖人，有的人可能只喜欢勇士，也有的人喜欢骑士、绿军、猛龙等一大堆。对于这种特征，我们本文将其称为多值离散特征。

根据DeepFM的思想，我们需要将每一个field的特征转换为定长的embedding，即使有多个取值，也是要变换成定长的embedding。

那么，一种思路来了，比如一个用户喜欢两个球队，这个field的特征可能是[1,1,0,0,0,0,0.....0]，那么我们使用两次embedding lookup，再取个平均不就好了嘛。

嗯，这的确也许可能是一种思路吧，在tensorflow中，其实有一个函数能够实现我们上述的思路，那就是tf.nn.embedding_lookup_sparse。别着急，我们一步一步来实现多值离散特征的embedding处理过程

# 输入数据

假设我们有三条数据，每条数据代表一个user所喜欢的nba球员，比如有登哥，炮哥，杜老四，慕斯等等：

In [4]:
import tensorflow as tf

In [1]:
csv = [
  "1,harden|james|curry",
  "2,wrestbrook|harden|durant",
  "3,|paul|towns",
]
TAG_SET = ["harden", "james", "curry", "durant", "paul","towns","wrestbrook"]


# 数据处理
这里我们需要一个得到一个SparseTensor，即多为稀疏矩阵的一种表示方式，我们只记录非0值所在的位置和值。

比如说，下面就是我们对上面数据处理过后的一个SparseTensor，indices是数组中非0元素的下标，values跟indices一一对应，表示该下标位置的值，最后一个表示的是数组的大小。

In [2]:
def sparse_from_csv(csv):
  ids, post_tags_str = tf.decode_csv(csv, [[-1], [""]])
  table = tf.contrib.lookup.index_table_from_tensor(
      mapping=TAG_SET, default_value=-1) ## 这里构造了个查找表 ##
  split_tags = tf.string_split(post_tags_str, "|")
  return tf.SparseTensor(
      indices=split_tags.indices,
      values=table.lookup(split_tags.values), ## 这里给出了不同值通过表查到的index ##
      dense_shape=split_tags.dense_shape)

# 定义embedding变量

In [5]:
TAG_EMBEDDING_DIM = 3
embedding_params = tf.Variable(tf.truncated_normal([len(TAG_SET), TAG_EMBEDDING_DIM]))

In [6]:
tags = sparse_from_csv(csv)
embedded_tags = tf.nn.embedding_lookup_sparse(embedding_params, sp_ids=tags, sp_weights=None)

W0319 18:40:49.987010 18748 deprecation.py:323] From d:\python\lib\site-packages\tensorflow\python\ops\lookup_ops.py:1347: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0319 18:40:49.994989 18748 tf_logging.py:161] The default value of combiner will change from "mean" to "sqrtn" after 2016/11/01.



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



sp_ids就是我们刚刚得到的SparseTensor，而sp_weights=None代表的每一个取值的权重，如果是None的话，所有权重都是1，也就是相当于取了平均。如果不是None的话，我们需要同样传入一个SparseTensor，代表不同球员的喜欢权重。

# 测试结果

In [7]:
with tf.Session() as s:
  s.run([tf.global_variables_initializer(), tf.tables_initializer()])
  print(s.run([embedded_tags]))

[array([[-0.06959277,  0.75110626,  1.1484662 ],
       [-0.95036954,  0.72681165,  0.7044843 ],
       [-0.01936512, -0.16457653,  0.2063216 ]], dtype=float32)]
